In [1]:
import torch
import os
import sys
from pathlib import Path
sys.path.append(os.getcwd())
sys.path.append(str(Path(os.getcwd()).parent))
import matplotlib.pyplot as plt
import numpy as np
sys.path.append('../utils/')
from configuration import *
from utils import *
import FigureConfig as FC
import pNN_Power_Aware as pNN

args = parser.parse_args([])

In [6]:
dataset_num = 13
act = 'sig'

In [7]:
result = torch.load(f'{dataset_num}_{act}_result.matrix')
result.shape

torch.Size([10, 4, 13, 8])

In [8]:
import numpy as np

target = [[ 200.,  240.,  400.,  500.,  300.,  300.,  200.,  300.,  600.,  280.,  340.,  280.,  240.],
          [ 400.,  480.,  800., 1000.,  600.,  600.,  400.,  600., 1200.,  560.,  680.,  560.,  480.],
          [ 600.,  720., 1200., 1500.,  900.,  900.,  600.,  900., 1800.,  840., 1020.,  840.,  720.],
          [ 800.,  960., 1600., 2000., 1200., 1200.,  800., 1200., 2400., 1120., 1360., 1120.,  960.]]
target = np.array(target)

In [9]:
def calculate_hessian_eigenvalues(model, dataloader, loss_function):
    eigenvalues = {}

    model.train()
    
    for images, labels in dataloader:
        
        # Forward pass
        outputs = model(images)
        loss = loss_function(model, images, labels)
        
        # Compute gradients
        model.zero_grad()
        loss.backward(create_graph=True)
        
        for n, p in model.named_parameters():
            if p.requires_grad:
                grad = p.grad.view(-1)
                hessian = torch.zeros(grad.size(0), grad.size(0))
                # Compute each element of the Hessian
                for i in range(len(grad)):
                    grad2 = torch.autograd.grad(grad[i], p, retain_graph=True, allow_unused=True)[0].view(-1)
                    if grad2 is not None:
                        hessian[i, :] = grad2
                
                # Calculate the eigenvalues of the Hessian
                eigvals = torch.linalg.eigvals(hessian)
                eigenvalues[n] = eigvals
    
    return eigenvalues

In [10]:
values = []

for ds in range(dataset_num):
    values_ds = []
    
    args.DATASET = ds
    train_loader, datainfo = GetDataLoader(args, 'train', path='../dataset/')
    
    
    print(datainfo)

    
    
    for i in range(4):
        values_target = []
        
        power = result[:,i,ds,3]
        acc = result[:,i,ds,2]

        best_acc = acc.max()
        seed = torch.where(acc == best_acc)[0][0].item()

        
        args.powerbalance = target[i,ds]
        args.POWER = target[i,ds]

        
        setup = f"data_{datainfo['dataname']}_seed_{seed}_Penalty_AL_Factor_{int(args.powerbalance):04d}"
        print(setup)

        model_PT_name = f'./models/pNN_{setup}.model'
        model_PT = torch.load(model_PT_name)
        model_PT.pruning

        args.powerestimator = 'AL'
        args.powerbalance = 0.
        loss_function = pNN.Lossfunction(args)

        # Calculate the eigenvalues of the Hessian matrix
        eigenvalues = calculate_hessian_eigenvalues(model_PT, train_loader, loss_function)
        
        # Append the top 3 eigenvalues for each parameter to values_target
        for n, eigvals in eigenvalues.items():
            # Sort eigenvalues by their absolute values in descending order
            top_eigvals = sorted(eigvals, key=lambda x: abs(x), reverse=True)[:3]
            print(f'Top 3 eigenvalues for Hessian of {n}:')
            for eigval in top_eigvals:
                values_target.append(abs(eigval).item())
                print(abs(eigval))

        values_ds.append(values_target)
    values.append(values_ds)

{'dataname': 'acuteinflammation', 'N_feature': 6, 'N_class': 2, 'N_train': 70, 'N_valid': 23, 'N_test': 25}
data_acuteinflammation_seed_3_Penalty_AL_Factor_0200


/home/kit/itec/qc0876/miniconda3/envs/train-model/lib/python3.12/site-packages/torch/autograd/__init__.py:266: UserWarning: Using backward() with create_graph=True will create a reference cycle between the parameter and its gradient which can cause a memory leak. We recommend using autograd.grad when creating the graph to avoid this. If you have to use this function, make sure to reset the .grad fields of your parameters to None after use to break the cycle and avoid the leak. (Triggered internally at /opt/conda/conda-bld/pytorch_1708025824022/work/torch/csrc/autograd/engine.cpp:1177.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Top 3 eigenvalues for Hessian of act.rt_:
tensor(0.)
tensor(0.)
tensor(0.)
Top 3 eigenvalues for Hessian of inv.rt_:
tensor(0.)
tensor(0.)
tensor(0.)
Top 3 eigenvalues for Hessian of model.0-th pLayer.theta_:
tensor(0.)
tensor(0.)
tensor(0.)
Top 3 eigenvalues for Hessian of model.1-th pLayer.theta_:
tensor(0.)
tensor(0.)
tensor(0.)
data_acuteinflammation_seed_2_Penalty_AL_Factor_0400
Top 3 eigenvalues for Hessian of act.rt_:
tensor(4.6674e-06)
tensor(7.8364e-07)
tensor(2.0618e-10)
Top 3 eigenvalues for Hessian of inv.rt_:
tensor(1.2237e-13)
tensor(8.0916e-14)
tensor(2.9229e-14)
Top 3 eigenvalues for Hessian of model.0-th pLayer.theta_:
tensor(1.6756e-10)
tensor(6.0326e-11)
tensor(3.8695e-13)
Top 3 eigenvalues for Hessian of model.1-th pLayer.theta_:
tensor(3.6432e-10)
tensor(3.2191e-14)
tensor(9.3578e-16)
data_acuteinflammation_seed_0_Penalty_AL_Factor_0600
Top 3 eigenvalues for Hessian of act.rt_:
tensor(8.0781e-05)
tensor(1.6437e-05)
tensor(4.3905e-10)
Top 3 eigenvalues for Hessian o

In [11]:
values = torch.tensor(values)
top_values, top_indices = values.topk(6, dim=-1)

# top_values will have the shape [dataset_num, 4, 3]
print("Shape of the resulting tensor:", top_values.shape)

Shape of the resulting tensor: torch.Size([13, 4, 6])


In [12]:
torch.save(top_values, f'../RESULT/{dataset_num}_{act}_al.hessian')